In [1]:
import pandas as pd
import numpy as np
import warnings
from scipy.sparse import csr_matrix
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
anime_data_cleaned = pd.read_csv('csv/2020/anime_2020_cleaned.csv')
rating_data = pd.read_csv('csv/2020/rating_2020.csv')

In [96]:
rating_data.shape

(57633278, 3)

In [114]:
rating_data.columns

Index(['user_id', 'MAL_ID', 'rating'], dtype='object')

In [3]:
counts = rating_data['user_id'].value_counts()
rating_data = rating_data[rating_data['user_id'].isin(counts[counts >= 200].index)]
rating_data.rename(columns={'anime_id':'MAL_ID'}, inplace=True)

In [13]:
rating_data.shape

(41320341, 3)

In [4]:
anime_rating_data = anime_data_cleaned.merge(rating_data, on='MAL_ID', how='inner')

In [2]:
# anime_rating_data = pd.read_csv('csv/2020/anime_rating_2020.csv')

In [3]:
anime_rating_data.shape

(38541711, 12)

## User-based collaborative filtering - v1.0
This is a basic user-based collaborative filtering algorithm. 
It uses the cosine similarity to find the most similar users to a given user. To reduce the time of cosine similarity computation, it uses the sparse matrix and clustering.

The `recommend_items_in_cluster` funciton takes user_id as input and returns recommended items based on the predicted ratings.

In [3]:
user_pivot = pd.read_csv('csv/2020/user_pivot_2020.csv')

In [107]:
user_pivot.shape

(95081, 4687)

In [4]:
from scipy.sparse import csr_matrix

#Creating a sparse matrix
user_anime_matrix = csr_matrix(user_pivot.values)

In [5]:
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
optimal_k = 6  # Replace with the optimal number of clusters determined in the previous step
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
user_clusters = kmeans.fit_predict(user_anime_matrix)

In [7]:
def recommend_items_in_cluster(user_id, k=10):
    user_index = user_pivot.index.get_loc(user_id)
    cluster_id = user_clusters[user_index]
    users_in_cluster = np.where(user_clusters == cluster_id)[0]
    print('-------user is in cluster------')

    cluster_similarity = cosine_similarity(user_anime_matrix[users_in_cluster])
    norm = np.abs(cluster_similarity).sum(axis=1)
    predicted_ratings = cluster_similarity.dot(user_pivot.iloc[users_in_cluster].values) / norm[:, np.newaxis]
    print('-------predicted ratings------')

    unrated_items = np.where(user_pivot.iloc[user_index] == 0)[0]
    item_indices = np.argsort(predicted_ratings[user_index, unrated_items])[-k:][::-1]
    recommended_item_ids = user_pivot.columns[unrated_items[item_indices]]
    return recommended_item_ids

In [8]:
recommend_items_in_cluster(6, k=10)

-------users in cluster------
-------predicted ratings------


Index(['Fullmetal Alchemist: Brotherhood', 'Code Geass: Hangyaku no Lelouch',
       'One Punch Man', 'Code Geass: Hangyaku no Lelouch R2',
       'Boku no Hero Academia', 'Re:Zero kara Hajimeru Isekai Seikatsu',
       'Hataraku Maou-sama!', 'Kono Subarashii Sekai ni Shukufuku wo!',
       'Fate/Zero', 'Shigatsu wa Kimi no Uso'],
      dtype='object')

## User-based collaborative filtering - v2.0

Split dataset into train and test

In [5]:
anime_ratings = anime_rating_data[['user_id', 'MAL_ID', 'rating']]

In [6]:
anime_ratings.shape

(26724793, 3)

### Split data into train and test

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets based on the users
train_data, test_data = train_test_split(rating_data, test_size=0.2, random_state=42)

In [5]:
user_item_matrix = train_data.pivot_table(index='user_id', columns='MAL_ID', values='rating').fillna(0)
train_data_sparse = csr_matrix(user_item_matrix.fillna(0).values)

In [100]:
user_item_matrix

MAL_ID,1,5,6,7,8,15,16,17,18,19,...,47398,47402,47403,47410,47614,47616,47618,48177,48456,cluster
user_id,,,,,,,,,,,,,,,,,,,,,
3,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
6,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
19,8.0,6.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353385,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
353386,7.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
353392,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [98]:
train_data_sparse

<95143x16847 sparse matrix of type '<class 'numpy.float64'>'
	with 33056272 stored elements in Compressed Sparse Row format>

### Clustering users with KMeans

In [6]:
from sklearn.cluster import KMeans

# Cluster the users based on the sparse matrix
kmeans = KMeans(n_clusters=10, random_state=42).fit(train_data_sparse)

# Add the cluster labels to the user-item matrix
user_item_matrix['cluster'] = kmeans.labels_

### Predict ratings for the specific items that the user has rated in the test set

In [165]:
# pick random user_id from test_data
user_id = test_data['user_id'].sample(1).values[0]
user_id

55604

In [194]:
user_id = 353385

In [93]:
anime_ids = test_data.loc[test_data['user_id']==user_id]['MAL_ID'].values

In [196]:
user_item_matrix.index.get_loc(user_id)

95138

In [229]:
train_data_sparse.shape

(95143, 16847)

In [238]:
train_data_sparse[95138, :].shape

(1, 16847)

In [376]:
rating_data['MAL_ID'].nunique()

16864

In [360]:
def predict_anime_ratings(user_id):

    anime_ids = test_data.loc[test_data['user_id'] == user_id]['MAL_ID'].values
    # Get the cluster of the user
    user_cluster = user_item_matrix['cluster'].loc[user_item_matrix['cluster'].index == user_id].values[0]
    
    # Get the users in the same cluster
    users_in_cluster = user_item_matrix[user_item_matrix['cluster'] == user_cluster].index.intersection(train_data_sparse[:, 0].toarray().flatten().astype(int))
    
    # Get the ratings of the users in the same cluster
    ratings_of_users_in_cluster = train_data_sparse[users_in_cluster]
    
    # Calculate the similarity between the user and the users in the same cluster
    similarity = cosine_similarity(train_data_sparse[user_id], ratings_of_users_in_cluster)
    
    # Get the ratings for the specified anime_ids
    anime_ratings = np.asarray(train_data_sparse[users_in_cluster, :][:, anime_ids].todense())
    
    # Get the top-k similar users
    top_similar_users = np.argsort(similarity)[0][-1]
    
    # Calculate the predicted ratings for the specified anime_ids
    predicted_ratings = anime_ratings.dot(similarity[0][top_similar_users]) / np.sum(np.abs(similarity[0][top_similar_users]))
    
    return predicted_ratings


In [413]:
from sklearn.metrics.pairwise import cosine_similarity
def predict_anime_ratings_v2(user_id):
    # Get the anime_ids that the user has actually rated
    anime_ids = test_data.loc[test_data['user_id'] == user_id]['MAL_ID'].values

    # Get the cluster of the user (e.g., 5)
    user_cluster = user_item_matrix['cluster'].loc[user_item_matrix['cluster'].index == user_id].values[0]
    
    # Get the users in the same cluster (e.g., [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
    users_in_cluster = user_item_matrix[user_item_matrix['cluster'] == user_cluster].index
    
    # Get the ratings of the users in the same cluster
    ratings_of_users_in_cluster = train_data_sparse[user_item_matrix.index.isin(users_in_cluster)]
    
    row_index = user_item_matrix.index.get_loc(user_id)

    # Calculate the similarity between the user and the users in the same cluster
    similarity = cosine_similarity(train_data_sparse[row_index], ratings_of_users_in_cluster)

    # Get the ratings for the specified anime_ids that exist in train_data_sparse
    # It is actually searching it from user_item_matrix but its fine because train_data_sparse is a csr_matrix made from user_item_matrix
    common_anime_ids = set(anime_ids).intersection(set(user_item_matrix.index.values))
    common_anime_ids = np.array(sorted(list(common_anime_ids)))
    
    common_anime_ids_idx  = [user_item_matrix.columns.get_loc(anime_id) for anime_id in common_anime_ids]
    
    # Use this in case if you want to get all the anime_ids from train_data_sparse
    anime_ids_idx  = [user_item_matrix.columns.get_loc(anime_id) for anime_id in anime_ids]

    # get index of users_in_cluster, common_anime_ids
    users_in_cluster_idx = [user_item_matrix.index.get_loc(user_id) for user_id in users_in_cluster]

    # Get the ratings for the specified anime_ids
    anime_ratings = np.asarray(train_data_sparse[users_in_cluster_idx, :][:, anime_ids_idx].todense())

    # Get the top-k similar users
    top_similar_users = np.argsort(similarity)[0][-1]
    
    # Calculate the predicted ratings for the specified anime_ids
    predicted_ratings = anime_ratings.dot(similarity[0][top_similar_users]) / np.sum(np.abs(similarity[0][top_similar_users]))
    
    return predicted_ratings[0]


In [414]:
predicted_ratings = predict_anime_ratings_v2(user_id)
predicted_ratings, predicted_ratings.shape

(array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  9.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         8.,  0.,  0.,  0.,  0.,  7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,  0.,  0.,
         0.,  9.,  0.,  0.,  0.,  0.,  0.,  0.,  7.,  0.,  0.,  0.,  0.]),
 (65,))

In [411]:
actual_ratings = test_data.loc[test_data['user_id']==user_id]['rating'].values
actual_ratings, actual_ratings.shape

(array([ 8,  6,  6,  9,  6,  5,  7,  8, 10,  5,  9,  5,  8,  7,  7,  7,  9,
         8,  9,  6,  8,  6,  6,  9,  7,  7,  9,  7,  5,  8,  6,  8,  1,  6,
         7,  7,  9,  9,  9,  8,  6,  6,  7,  8,  9,  8,  8,  5,  8,  5,  7,
         8,  3,  8,  7,  6,  7,  8,  7,  8,  8,  9,  7,  7,  2]),
 (65,))

In [384]:
def calculate_mae(predicted_ratings, actual_ratings):
    return np.mean(np.abs(predicted_ratings - actual_ratings))

In [385]:
calculate_mae(predicted_ratings, actual_ratings)

6.384615384615385

In [409]:
common_anime_ids = set(anime_ids).intersection(set(user_item_matrix.index.values))
common_anime_ids = np.array(sorted(list(common_anime_ids)))

In [393]:
test_data_for_user = test_data.loc[test_data['user_id']==user_id]
actual_ratings_for_real = test_data_for_user.loc[test_data_for_user['MAL_ID'].isin(common_anime_ids)]['rating'].values
actual_ratings_for_real

array([6, 7, 9, 8, 7, 8, 9, 6, 6, 7, 9, 9, 9, 8, 5, 3, 6, 7, 8, 7])

### Predict ratings for top_k recommended items

In [185]:
train_data_sparse.shape

(95143, 16847)

In [175]:
def predict_ratings(user_id, k):
    # Get the cluster of the user
    # user_cluster = user_item_matrix.loc[user_id, 'cluster']
    user_cluster = user_item_matrix['cluster'].loc[user_item_matrix['cluster'].index == user_id].values[0]
    
    # Get the users in the same cluster
    users_in_cluster = user_item_matrix[user_item_matrix['cluster'] == user_cluster].index.intersection(train_data_sparse[:, 0].toarray().flatten().astype(int))
    
    # Get the ratings of the users in the same cluster
    ratings_of_users_in_cluster = train_data_sparse[users_in_cluster]
    
    # Calculate the similarity between the user and the users in the same cluster
    similarity = cosine_similarity(train_data_sparse[user_id], ratings_of_users_in_cluster)
    
    # Get the top-k similar users
    top_k_similar_users = np.argsort(similarity)[0][-k:]
    
    # Get the items rated by the top-k similar users
    items_rated_by_top_k_users = train_data_sparse[top_k_similar_users].toarray().T
    
    # Get the items not rated by the user
    unrated_items_by_user = np.isnan(user_item_matrix.loc[user_id])
    
    # Calculate the predicted rating for the unrated items
    predicted_ratings = items_rated_by_top_k_users.dot(similarity[0][top_k_similar_users]) / np.sum(similarity[0][top_k_similar_users])
    
    # Get the top-k recommended items
    top_k_items = np.argsort(predicted_ratings)[-k:]
    
    # Get the predicted ratings for the top-k items
    predicted_ratings_for_top_k_items = predicted_ratings[top_k_items]
    
    return top_k_items, predicted_ratings_for_top_k_items


In [181]:
# Top 10 recommendation and predicted ratings for user 6
top_k, predicted_ratings = predict_ratings(353385, k=10)
top_k, predicted_ratings

IndexError: row index (353385) out of range

### Evaluation

#### MAE@K

In [78]:
# Here is the problem, none of top_k animes are in the test_data
# So it doesn't work to calculate MAE by comparing train and test data
test_data.loc[test_data['user_id']==6].sort_values(by='MAL_ID', ascending=False)

,user_id,MAL_ID,rating
739,6,13675,5
912,6,11691,7
759,6,11099,8
681,6,11073,6
766,6,10622,8
825,6,10497,9
677,6,10161,8
773,6,10067,8
785,6,9846,7
703,6,9756,10


#### MAP

In [11]:
def calculate_MAP_k(user_id, k=10):
    # Get the items in the test set for the user
    test_items = test_data.loc[user_id].dropna().index
    
    # Get the top-k recommended items
    top_k_items = get_top_k_recommendations(user_id, k)
    
    # Get the indices of the recommended items that are in the test set
    relevant_items = np.intersect1d(top_k_items, test_items, assume_unique=True)
    
    # Calculate the precision at k
    precision_at_k = len(relevant_items) / k
    
    return precision_at_k